In [1]:
from cdnsim.arrival import Arrival


class Constant(Arrival):
    """
    Constant arrival process. *rate* number of arrival in every tick.
    """
    def __init__(self, rate: float, ticks: int, **kwargs):
        super().__init__(**kwargs)
        self.__rate = rate
        self.__ticks = ticks

    def __next__(self) -> int:
        if self.__ticks <= 0:
            raise StopIteration
        self.__ticks -= 1
        return int(self.__rate)


In [2]:
from typing import Self

import pandas as pd
from scipy.stats import randint
import numpy as np

from cdnsim.requests.requests import BaseRequests


class Uniform(BaseRequests):
    """
    Implements a Uniform distribution based request profile.
    """

    def __init__(self, cbase: int):
        """
        :param cbase: number of content
        """
        super().__init__([],{'content':[]})
        self._cbase = cbase

    def generate(self, k: int) -> Self:
        r = np.unique(randint.rvs(1, self._cbase + 1, size=k), return_counts=True)
        return BaseRequests(freqs=r[1], index={'content': r[0]})


In [3]:
from cdnsim.cache import Cache
from cdnsim.requests import BaseRequests


class NonCache(Cache):
    """
    For demonstration purposes, a non caching cache implementation.
    """
    def _process_ingress(self, requests: BaseRequests) -> None:
        # do not cache, simply forward all requests via a round-robin remote selection
        for remote, request in zip(self.remotes, requests / len(self.remotes)):
            self._send(remote, request)


In [ ]:
from cdnsim import Client, Origin
# create the nodes
client = Client(arrival=Constant(rate=50, ticks=5), requests=Uniform(cbase=20))
cache = NonCache()
origin = Origin()

# establish connections
client.connect_to(cache)
cache.connect_to(origin)

# run simulation
try:
    client.start_all()
    client.join_all()
except KeyboardInterrupt:
    pass
finally:
    client.terminate()

# get stats
...


In [4]:
# %load constant.py
from cdnsim.arrival import Arrival


class Constant(Arrival):
    """
    Constant arrival process. *rate* number of arrival in every tick.
    """
    def __init__(self, rate: float, ticks: int, **kwargs):
        super().__init__(**kwargs)
        self.__rate = rate
        self.__ticks = ticks

    def __next__(self) -> int:
        print(f"IIIIII {self.__ticks}")
        if self.__ticks <= 0:
            raise StopIteration
        self.__ticks -= 1
        return int(self.__rate*2)


In [5]:
print(1)

1
